# Definición del problema

Se desea entrenar un modelo que sea capaz de 'completar' una palabra a medio escribir, o proponer una corrección para una palabra ya escrita en caso de que la misma se encuentre mal escrita.

Se utilizará un algoritmo de 'hallar la palabra incorrecta' para determinar si una palabra está escrita incorrectamente, de acuerdo a un lexicón construido con palabras extraídas de la página web de la RAE (disponible en https://github.com/JorgeDuenasLerin/diccionario-espanol-txt, actualizado en Mayo 2024)

Además, se construirá una matriz de probabilidad con las palabras extraídas para que las recomendaciones de completado y corrección se realicen en función de la frecuencia de utilización de las palabras. El sistema será capaz de realizar estas funciones en Español.

Se utilizarán textos para entrenarlo.

# Estructura del modelo

# Datos de entrada

In [2]:
#Extrayendo el texto
txt_file = "text_dump.txt"

lines: list[str] = []
# complete_text: str = ""
with open(txt_file, "r") as file:
  for line in file:
    if line != "\n":
      lines.append(line)
      # complete_text += line

# print(len(lines))
# print(complete_text)

In [3]:
#Formando la data de entrada y salida esperada
complete_text: str = ""

for paragraph in lines[31 : 14346]:
  complete_text += paragraph

#Construyendo la data de input, output y conjunto de tokens
encoder_tokens: dict[str, int] = dict()
decoder_tokens: dict[str, int] = dict()
# input_chars: list[str] = []
# output_chars: list[str] = []
input_words: list[str] = []
output_words: list[str] = []


# input_chars.append(complete_text[0])

#Llenar los inputs y outputs de manera escalonada
input_word = ""
# output_word = ""
current_char = ""
index = 0
while current_char != "\n" and current_char != " ":
  current_char = complete_text[index]
  input_word += current_char
  index += 1

  if current_char not in encoder_tokens:
    tk_i = len(encoder_tokens)
    encoder_tokens[current_char] = tk_i
    decoder_tokens[tk_i] = current_char

input_words.append(input_word)
input_word = ""
for i in range(index, len(complete_text) - 1):
  current_char = complete_text[i]

  if current_char != " " and current_char != "\n":
    input_word += current_char
  else:
    input_words.append(input_word + current_char)
    output_words.append(input_word + current_char)
    input_word = ""

  if current_char not in encoder_tokens:
    tk_i = len(encoder_tokens)
    encoder_tokens[current_char] = tk_i
    decoder_tokens[tk_i] = current_char

output_words.append(input_word)

# for i in range(0, len(complete_text) - 1):
#   input_char = complete_text[i]

#   input_chars.append(input_char)
#   output_chars.append(complete_text[i + 1])

#   if input_char not in encoder_tokens:
#     index = len(encoder_tokens)
#     encoder_tokens[input_char] = index
#     decoder_tokens[index] = input_char


# index = 0
# for pair in zip(input_chars, output_chars):
#   print(pair)
#   index += 1
#   if index >= 100:
#     break
# for key, value in zip(encoder_tokens.keys(), encoder_tokens.values()):
#   print(f"Key: {key} | Value: {value}")

In [4]:
import numpy as np
# Creando la matriz de caracteres

input_size = len(input_words)
max_input_word_length = max([len(word) for word in input_words])
max_output_word_length = max([len(word) for word in output_words])
token_size = len(encoder_tokens)

encoder_input_data = np.zeros((input_size, max_input_word_length, token_size), dtype= "float32")
decoder_input_data = np.zeros((input_size, max_output_word_length, token_size), dtype= "float32")
decoder_output_data = np.zeros((input_size, max_output_word_length, token_size), dtype= "float32")

for i, (input, output) in enumerate(zip(input_words, output_words)):
  for j, char in enumerate(input):
    encoder_input_data[i, j, encoder_tokens[char]] = 1.0 #Se guardan las palabras de input

  for j, char in enumerate(output):
    decoder_input_data[i, j, encoder_tokens[char]] = 1.0 #Se guarda la siguiente palabra esperada para cada input
    if j > 0:
      decoder_output_data[i, j - 1, encoder_tokens[char]] = 1.0 #Se guarda en la salida del decodificador la palabra esperada en tiempo t + 1

# for i, (input, output) in enumerate(zip(input_chars, output_chars)):
#   input_data[i, 0,encoder_tokens[input]] = 1.0
#   output_data[i, 0,encoder_tokens[output]] = 1.0

In [ ]:
# print(encoder_input_data[0])
# for n in decoder_output_data[0]:
#   print(n)
# print(decoder_output_data[0])

# Construción del modelo

In [5]:
import tensorflow as tf
from tensorflow import keras
from keras import initializers

neurons = 400

In [1]:
weight_initializer = initializers.RandomNormal(mean= 0.05, stddev= 0.04)

encoder_inputs = keras.Input(shape= (None, token_size))
encoder = keras.layers.LSTM(neurons, return_state= True, kernel_initializer= weight_initializer)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape= (None, token_size))

decoder_lstm = keras.layers.LSTM(neurons, return_sequences= True, return_state= True, kernel_initializer= weight_initializer)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state= encoder_states)
decoder_dense = keras.layers.Dense(token_size, activation= "softmax", kernel_initializer= weight_initializer )
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer= "rmsprop", loss= "categorical_crossentropy", metrics= ["accuracy"])

NameError: name 'initializers' is not defined

# Entrenamiento del modelo

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_output_data, batch_size= 512, epochs= 5, validation_split= 0.3)

model.save("s2s-Archivo.keras")

# Evaluación

In [67]:
model = keras.models.load_model("s2s-Archivo.keras")

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_inputs = [keras.Input(shape= (neurons,)), keras.Input(shape= (neurons,))]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state= decoder_state_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

def decode_word(input_char: str) -> str:
  data = np.zeros((1, 1, token_size))
  data[0, 0, encoder_tokens[input_char]] = 1.0

  state_value = encoder_model.predict(data)

  target_sequence = np.zeros((1, 1, token_size))
  target_sequence[0,0, encoder_tokens[input_char]] = 1.0

  decoded_word = input_char
  sampled_char = ""
  failsafe = 0
  while sampled_char != " " and failsafe < 25:
    failsafe += 1
    output_tokens, h, c = decoder_model.predict([target_sequence] + state_value)

    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = decoder_tokens[sampled_token_index]
    decoded_word += sampled_char

    target_sequence = np.zeros((1, 1, token_size))
    target_sequence[0, 0, sampled_token_index] = 1.0

    state_value = [h, c]

  return decoded_word


# sequence = input_data[1 : 2]
# print(decoder_tokens[np.argmax(input_data[1, -1, :])])

# # Define the shape of the decoder input explicitly
# decoder_input_shape = (sequence.shape[0], sequence.shape[1], token_size)
# # Assuming token_size is defined in your code

# # Create the decoder input with the defined shape
# decoder_input = output_data[1 : 2]
# i = 0
# decoded_char = ""
# while decoded_char != " " and i < 25:
#   i += 1
#   # Now, predict using the model
#   output_tokens = model.predict([sequence, decoder_input])

#   token_index = np.argmax(output_tokens[0, -1, :])

#   decoded_char = decoder_tokens[token_index]

#   print(decoded_char)

#   sequence[0, 0, token_index] = 1.0

In [69]:
predicted_word = decode_word("K")

print(predicted_word)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
jooaaaaaaaaaaaaaaa 
